# REvIEW
## Equivalent width calculator
***R***outine for
***Ev***aluating and 
***I***nspecting
***E***quivalent
***W***idths


Based on postage40mpfit.pro by Dr David Yong.

This notebook should output a csv and a plot for every line you are fitting to.



### Relavent files

There are two files which contain all the functions.

```REvIEW_plots.py```: concerns any functions in charge of creating the plots. These funcitons call the fitting functions within the plotting functions.

```REvIEW_fitting.py```: contains all the fitting functions. This includes the gradient preprocessing algorithm and the variable gaussian fitting functions. 

## Imports

Add any python modules to ```my_imports.py```. 

There are two files which contain REvIEW functions. The only functions that are needed here are in the plotting module.

In [1]:
from my_imports import *
from REvIEW_plots import gen_plots, line_incrementer

### Directories

Define the location of the spectra (```spectra_path```) and line list (```line_list_path```). 

You will need to create a directory where you can put the ew .png and the output csv.


In [2]:
#line_list_path = '/Users/maddie/Desktop/M22/line lists'
spectra_path = '/Users/maddie/Desktop/M22/spectra'
ew_path = '/Users/maddie/Desktop/M22/equivelant widths/REvIEW 2'



# Steph's stars
line_list_path = '/Users/maddie/Documents/steph_daospec/line lists'
#spectra_path = '/Users/maddie/Documents/steph_daospec/StephSpecsTxt'
#ew_path = '/Users/maddie/Documents/steph_daospec/ref'

### Spectra files

Pass a list of file names (```file_names```) that will be analysed. These must have the same names that appear in the ```spectra_path``` direcotry.

***note:*** UVES spectra has both a blue and red arm so remember to change the names if you want to analyse the total data set.

Pass a list of the radial velovity corrections (```rv_correction```). The numbers must line up with the order that you listed your file names. 


In [3]:
#file_names_l = ['scomb2.M22_C.L.txt','scomb2.M22_III-3.L.txt','scomb2.M22_III-14.L.txt','scomb2.M22_III-15.L.txt','scomb2.M22_III-52.L.txt','scomb2.M22_IV-102.L.txt', 'reference_6752.txt']

file_names = ['scomb2.M22_C.U.txt','scomb2.M22_III-3.U.txt','scomb2.M22_III-14.U.txt','scomb2.M22_III-15.U.txt','scomb2.M22_III-52.U.txt','scomb2.M22_IV-102.U.txt', 'reference_6752.txt']

#file_names_l = ['NGC288-20CR.txt','NGC288-338R.txt','NGC288-281R.txt','NGC288-287R.txt','NGC288-344R.txt','NGC288-351R.txt', 'NGC288-403R.txt', 'NGC288-531R.txt']

#file_names_l = ['NGC288-20CB.txt','NGC288-338B.txt','NGC288-281B.txt','NGC288-287B.txt','NGC288-344B.txt','NGC288-351B.txt', 'NGC288-403B.txt', 'NGC288-531B.txt']

#file_names_l = ['NGC362-1137R.txt','NGC362-1334R.txt','NGC362-1401R.txt','NGC362-1423R.txt','NGC362-1441R.txt','NGC362-2127R.txt', 'NGC362-77R.txt', 'NGC362-MB2R.txt']

#file_names_l = ['RefStar.txt']

# M 22
rv_correction = [-178.3, -122.7, -178.3, -118.1, -158.0, -108.2, -36.5]
#rv_correction = [-36.277]

# NGC 288
#rv_correction = [-73.608, -51.457, -57.208, -56.192, -64.926, -63.324, -71.898, -68.111]

# NGC 362
#rv_correction = [226.846, 217.118, 232.262, 232.964, 228.204, 218.899, 227.229, 237.344]


## Line list
Add the name of the line list which appears in the ```line_list_path``` directory.

Rest of the code will import the line list. This assumes that there is no header lines in the file and the first two columns are the wavelength and species.

Some edits might need to be made if the line list has any header lines. 

In [ ]:
#linelist_name = "line.list20"
linelist_name = "stephlinelist2.dat"

# Import the line list
os.chdir(line_list_path)
linelist2 = pd.read_table(linelist_name, sep="\s+", header=0, usecols = [0, 1],
                            names = ['wavelength', 'species'])

## Main celll for running REvIEW

### Variables
Three variables must be set:

```n_spectra:``` the number of spectra you are fitting to. Usually it's the same length as your ```file_names``` array but setting it to 1 is good for debugging/testing.

```lower_line_bound:``` the code will take any lines in the line list which are *higher* than this bound.

```upper_line_bound:``` the code will take any lines in the line list which are *lower* than this bound.

*note:* best to look at <=8 stars per run of review otherwise the plots become difficult to read. 

### Making the CSV
Creates a new csv with the name ```fit_outputs.csv``` if one doesn't already exist.
The csv is closed at the very end of the cell. If the code crashes within the main loop, nothing will be written to the csv.

### Main loop
The plotting function ```gen_plots``` from ```REvIEW_plots.py``` is called in the main loop which iterates through all the lines within the line bounds.


In [5]:
# Set these three variables
n_spectra = 7
lower_line_bound = 6645
upper_line_bound = 6646

# navigate to the directory where you will store your data
os.chdir(ew_path)

# Prepare the csv
if os.path.exists('fit_outputs.csv'):
    #just open
    output_csv  = open('fit_outputs.csv', "a+")
else:
    # open and write the headder
    output_csv  = open('fit_outputs.csv', "a+")
    # csv headder
    output_csv.write('line,star_name,flux_ratio,centre,sigma,ew,chi2,d,flag\n')


# Main loop which runs REvIEW
for line_info in line_incrementer(linelist2, 6645, 6646):
    # if theres been an error and yoou need to debug
    #print('line ', line_info[0])
    gen_plots(n_spectra, file_names_l, rv_correction, output_csv, spectra_path, ew_path, line_info[0], line_info[1], line_info[2], line_info[3])

# close the file
output_csv.close()

line  6645.06
